
# Create JSONs with aggregated data from database

Queries database and gets aggregated data (possible grouped before by country).
Creates a single JSON that contains all this information.

In [1]:
%load_ext autoreload
%autoreload 2

from pymongo import MongoClient
import datetime
import numpy as np
import pandas as pd
import getpass
from aggregation_functions import * # get all functions used below to communicate with mongodb database
from utility_functions import *


Be sure you are connect on the **EPFL network** (connection at EPFL or via VPN)

Note on authentification:
* Your username-password pair has read-only credentials
* Use the admin user to insert, modify, or create indexes

In [3]:
database = 'ada-project'
user = input('MongoDB name: ') #maxmordig
password = getpass.getpass('MongoDB password: ')

MongoDB name: maxmordig
MongoDB password: ········


In [4]:
# Mongo Client and authentification
client = MongoClient('www.cocotte-minute.ovh', 27017)
db = client[database]
db.authenticate(user, password)
collection = db['recipes']
collection.count()

16242

In [5]:
# Number of recipes
serverMongoVersion = client.server_info()['version']
serverMongoVersion

'3.2.10'

# Example usage of the aggregation function

In [6]:
results = group_and_get_aggregate_of_field(collection, group_by_field_address='$ada-country', 
                                aggregate_field_address='$nutrition.calories.amount', 
                                group_by_field_output_name='country', 
                                aggregate_field_output_name='avgCalories', 
                                aggregationFunction = '$avg' 
                               )

df = pd.DataFrame.from_dict(results)
df = df.set_index(['country'])
df.head(3)

,avgCalories,nbRecipes
country,,
israeli,255.634929,24
bangladeshi,355.954350,14
colombian,374.666667,13


In [7]:
# Get global average
results = group_and_get_aggregate_of_field(collection, group_by_field_address='get_global_aggregation', 
                                aggregate_field_address='$nutrition.calcium.amount', 
                                group_by_field_output_name='DummyColumn', 
                                aggregate_field_output_name='avgCalories', 
                                aggregationFunction = '$avg' 
                               )

df1 = pd.DataFrame.from_dict(results)
df1.head(3)

,DummyColumn,avgCalories,nbRecipes
0,Global aggregation,118.40618,14488


In [8]:
results = collection.aggregate([
        
        { '$match': 
            {
                # filter out the recipes that don't have a country (because we want to compare it to the country average
                'ada-country': { "$not": {"$size": 0}, "$exists": True }
            }
        },
        { '$project': 
            {
                '_id': 0,
                'ada-country': "$ada-country",
            }    
        },
        
    ]
                     )

results = list(results)
results

[{'ada-country': ['us-recipes', 'cuban']},
 {'ada-country': ['italian']},
 {'ada-country': ['italian']},
 {'ada-country': ['german']},
 {'ada-country': ['canadian']},
 {'ada-country': ['polish', 'russian']},
 {'ada-country': ['us-recipes', 'italian']},
 {'ada-country': ['italian']},
 {'ada-country': ['italian']},
 {'ada-country': ['australian-and-new-zealander']},
 {'ada-country': ['indian']},
 {'ada-country': ['chinese']},
 {'ada-country': ['greek']},
 {'ada-country': ['mexican']},
 {'ada-country': ['italian']},
 {'ada-country': ['italian']},
 {'ada-country': ['italian']},
 {'ada-country': ['mexican']},
 {'ada-country': ['vietnamese']},
 {'ada-country': ['italian']},
 {'ada-country': ['thai']},
 {'ada-country': ['german']},
 {'ada-country': ['spanish']},
 {'ada-country': ['indian']},
 {'ada-country': ['mexican']},
 {'ada-country': ['canadian']},
 {'ada-country': ['puerto-rican']},
 {'ada-country': ['italian']},
 {'ada-country': ['italian']},
 {'ada-country': ['italian']},
 {'ada-count

In [9]:
oneRecipe = collection.find_one({'recipeID':47564})
type( oneRecipe['ada-city'] )

list

# Several Fields

To produce the JSONs, we have the following setup.

We are interested in several fields per grouping, e.g. 'avgCalories' and 'avgIronIntake' and the goal is to merge this information into a single dataframe, one row per group. For instance, if a group corresponds to all recipes in a country, the row associated to this group contains averages of some quantities along its columns. In other words, it is the same as above, but we have columns 'avgCalories', 'avgIronIntake' and so on.

To first get an idea of what fields we have, we search for a simple recipe and see its attributes.

In [10]:
oneRecipe = collection.find_one({'recipeID':47564})
oneRecipe

{'_id': ObjectId('5825054ace06e50446084706'),
 'adUnit': {'adKeys': [7,
   79,
   95,
   125,
   148,
   150,
   169,
   173,
   184,
   201,
   221,
   235,
   241,
   242,
   249,
   254,
   265,
   608],
  'adZone': 'recipes',
  'contentProviderId': 451,
  'networkCode': '3865',
  'site': 'ar.ios.apps.allrecipes.recipes'},
 'ada-city': [],
 'ada-continent': ['asian'],
 'ada-country': ['indian'],
 'ada-region': [],
 'ada-subcontinent': [],
 'ada-tags': ['world-cuisine',
  'asian',
  'indian',
  'main-dishes',
  'curry',
  'vegetarian'],
 'cookMinutes': 45,
 'description': 'This is a really easy and tasty Indian dish that is sure to stir up your taste buds. Delicious baingan bharta is ready to eat with pita bread, Indian naan, or rice.',
 'directions': [{'displayValue': 'Preheat oven to 450 degrees F (230 degrees C).',
   'ordinal': 1,
   'videoTimestamp': 0},
  {'displayValue': 'Place eggplant on a medium baking sheet. Bake 20 to 30 minutes in the preheated oven, until tender. Remove

Here is an easier view of the dict structure indented by '->-'.

In [11]:
show_fields(oneRecipe)

> submitter
> ->-followingCount: 0
> ->-itemType: Cook
> ->-country: None
> ->-name: Yakuta
> ->-followersCount: 0
> ->-photo
> ->-->-itemType: Photo
> ->-->-recipeTitle: None
> ->-->-description: None
> ->-->-photoDetailUrl: None
> ->-->-urls: [{'url': 'http://ima...
> ->-->-rawItemType: 10
> ->-->-title: None
> ->-userID: 0
> ->-favoritesCount: 0
> ->-brandedSourceID: 0
> ->-ratingsCount: 0
> ->-isPro: False
> ->-madeRecipesCount: 0
> ->-city: None
> ->-reciepesCount: 0
> ->-promotedBrandPixelTrackingUrl: None
> ->-profileUrl: 
> ->-reviewsCount: 0
> ->-region: None
> ->-handle: None
> ->-rawItemType: 11
> ->-personalRecipeSharedCount: 0
> ada-country: ['indian']
> description: This is a really eas...
> footnotes: [{'text': 'Tip', 'or...
> links
> ->-self
> ->-->-href: https://apps.allreci...
> ->-recipeUrl
> ->-->-href: http://allrecipes.co...
> ratingCount: 451
> title: Baingan Bharta (Eggp...
> recipeID: 47564
> cookMinutes: 45
> ada-region: []
> nutrition
> ->-calories
> ->-->-ha

We will now create two dataframes (combined into a single dict).
The second dataframe will group the recipes by city.

For each dataframe, we will have four columns per group as below (for city name)

city    nbRecipes     avgCalories    totalVitaminAIntake

The last two columns contain respectively the average calories and the total vitamin A among all recipes in that group (one aggregation function is 'avg', the other is 'sum').


In [12]:
# contains all parameters to pass to the aggregation function

# group bys
groupBy_fields_toQuery = [
    {
        # get global average
        'group_by_field_address': 'get_global_aggregation',
        'group_by_field_output_name': 'DummyColumn',
    },
    {
        'group_by_field_address': '$ada-city', # as it appears in the database
        'group_by_field_output_name': 'city', # as it appears in dataframe
    },
]

# keep in sync with above groupBysToQuery, keys in the dict of dataframes
dataframe_names = ['global', 'per_city']

# what to aggregate for
aggregation_fields_toQuery = [
    {
        'aggregate_field_address': '$nutrition.calories.amount', # which field to aggregate
        'aggregate_field_output_name': 'avg_calories', # output name in result dict
        'aggregationFunction': '$avg' # how to compute the data for each group
    },
    {
        'aggregate_field_address': '$nutrition.vitaminA.amount', 
        'aggregate_field_output_name': 'total_vitaminA_Intake', 
        'aggregationFunction': '$sum' 
    },
]

all_dataframes_small = get_dataframe_per_groupby(collection, groupBy_fields_toQuery, aggregation_fields_toQuery, dataframe_names)

Query 1/4 with arguments: {'group_by_field_output_name': 'DummyColumn', 'group_by_field_address': 'get_global_aggregation', 'aggregationFunction': '$avg', 'aggregate_field_address': '$nutrition.calories.amount', 'aggregate_field_output_name': 'avg_calories'}
Query 2/4 with arguments: {'group_by_field_output_name': 'DummyColumn', 'group_by_field_address': 'get_global_aggregation', 'aggregationFunction': '$sum', 'aggregate_field_address': '$nutrition.vitaminA.amount', 'aggregate_field_output_name': 'total_vitaminA_Intake'}
Query 3/4 with arguments: {'group_by_field_output_name': 'city', 'group_by_field_address': '$ada-city', 'aggregationFunction': '$avg', 'aggregate_field_address': '$nutrition.calories.amount', 'aggregate_field_output_name': 'avg_calories'}
Query 4/4 with arguments: {'group_by_field_output_name': 'city', 'group_by_field_address': '$ada-city', 'aggregationFunction': '$sum', 'aggregate_field_address': '$nutrition.vitaminA.amount', 'aggregate_field_output_name': 'total_vita

Here are the dataframes we get.

In [13]:
all_dataframes_small['global'].head(3)

,avg_calories,nbRecipes,total_vitaminA_Intake
DummyColumn,,,
Global aggregation,352.507614,14488,1.959432e+07


In [14]:
all_dataframes_small['per_city'].head(3)

,avg_calories,nbRecipes,total_vitaminA_Intake
city,,,
atlanta,424.314499,28,85023.393070
baltimore,282.584182,40,27968.763135
boston,411.666383,12,2768.165049


Let us save these dataframes.

In [15]:
import os

json_output_filename_small = '../website/data/aggregatedDataSmall.json'

os.makedirs(os.path.dirname(json_output_filename_small), exist_ok=True)
write_dataframes_to_json(json_output_filename_small, all_dataframes_small)

#loadedAllDfs = load_dataframes_from_json(json_output_filename)

In [16]:
# see content of file
!cat '../website/data/aggregatedDataSmall.json'

{
    "per_city": {
        "seattle": {
            "nbRecipes": 67,
            "avg_calories": 415.338245,
            "total_vitaminA_Intake": 138905.35251
        },
        "san-francisco": {
            "nbRecipes": 41,
            "avg_calories": 291.9175343171,
            "total_vitaminA_Intake": 48289.644985
        },
        "washington-dc": {
            "nbRecipes": 17,
            "avg_calories": 291.5897064706,
            "total_vitaminA_Intake": 9453.05414
        },
        "oakland": {
            "nbRecipes": 26,
            "avg_calories": 448.8614230769,
            "total_vitaminA_Intake": 39844.8466
        },
        "new-york-city": {
            "nbRecipes": 125,
            "avg_calories": 411.8799015702,
            "total_vitaminA_Intake": 171463.7429213
        },
        "cincinnati": {
            "nbRecipes": 12,
            "avg_calories": 424.625025,
            "total_vitaminA_Intake": 39608.794
        },
        "atlanta": {
            "nbRecip

# Get many aggregation fields

We have demonstrated how to put several aggregations into a single dataframe.

We must pick more aggregation fields and we now choose the fields that make sense.

In [17]:
# First get all keys in the database and a description (an example value for each key)
all_keys_with_description = get_all_field_addresses_with_description(oneRecipe, shortenLongStrings=False)
all_keys_with_description

{'$_id': '5825054ace06e50446084706',
 '$adUnit.adKeys': '[7, 79, 95, 125, 148, 150, 169, 173, 184, 201, 221, 235, 241, 242, 249, 254, 265, 608]',
 '$adUnit.adZone': 'recipes',
 '$adUnit.contentProviderId': '451',
 '$adUnit.networkCode': '3865',
 '$adUnit.site': 'ar.ios.apps.allrecipes.recipes',
 '$ada-city': '[]',
 '$ada-continent': "['asian']",
 '$ada-country': "['indian']",
 '$ada-region': '[]',
 '$ada-subcontinent': '[]',
 '$ada-tags': "['world-cuisine', 'asian', 'indian', 'main-dishes', 'curry', 'vegetarian']",
 '$cookMinutes': '45',
 '$description': 'This is a really easy and tasty Indian dish that is sure to stir up your taste buds. Delicious baingan bharta is ready to eat with pita bread, Indian naan, or rice.',
 '$directions': "[{'displayValue': 'Preheat oven to 450 degrees F (230 degrees C).', 'ordinal': 1, 'videoTimestamp': 0}, {'displayValue': 'Place eggplant on a medium baking sheet. Bake 20 to 30 minutes in the preheated oven, until tender. Remove from heat, cool, peel, an

In [18]:
str.isnumeric('[7, 79, 95, 125, 148]') # false
str.isnumeric('45') # true
is_number(all_keys_with_description['$nutrition.potassium.amount']) # True
is_number('[1,2]') # False

False

We pick the fields that are numbers (and hence can be computed in an average or sum).

In [19]:
def get_keys_to_take_with_args(all_keys_with_description):
    # input: the keys with description (description = example of value of key)
    # finds all keys that are number fields, removes certain useless keys (e.g. ID)
    # then returns a dict with these keys as keys and each entry referring to 
    # the argument list to pass to the aggregate query
    
    accepted_keys_with_arguments = dict() # arguments for function

    for keyName, value in all_keys_with_description.items():

        # only recognizes single integers, not lists of integers (in strings) as wished
        if is_number(value):
              
            # ignore some keys
            if (keyName.startswith(tuple(['$adUnit', '$photo.']))
                or keyName.endswith(tuple(['.displayValue']))
                or (keyName in [])
                or any((word in keyName) for word in ['.rawItemType', 'ID','percentDailyValue'])):                
                
                continue

            aggregFunction = None
            if (keyName.endswith(tuple(['Count', '.amount']))
                or (keyName in ['$prepMinutes', '$cookMinutes', '$servings', '$readyInMinutes',
                                '$ratingAverage', '$similarRecipes.metaData.pagesize', 
                                '$similarRecipes.metaData.page'
                               ])):
                aggregFunction = '$avg'
                outputName = 'avg_' + make_column_name(keyName)
            else:
                print("Ignoring ok? {}: {}".format(keyName, value))
                continue

            args = {
                'aggregate_field_address': keyName, 
                'aggregate_field_output_name': outputName,
                'aggregationFunction': aggregFunction
            }
            accepted_keys_with_arguments[keyName] = args
    
    return accepted_keys_with_arguments

accepted_keys_with_arguments = get_keys_to_take_with_args(all_keys_with_description)
accepted_keys_with_arguments

{'$cookMinutes': {'aggregate_field_address': '$cookMinutes',
  'aggregate_field_output_name': 'avg_cookMinutes',
  'aggregationFunction': '$avg'},
 '$nutrition.calcium.amount': {'aggregate_field_address': '$nutrition.calcium.amount',
  'aggregate_field_output_name': 'avg_nutrition_calcium_amount',
  'aggregationFunction': '$avg'},
 '$nutrition.calories.amount': {'aggregate_field_address': '$nutrition.calories.amount',
  'aggregate_field_output_name': 'avg_nutrition_calories_amount',
  'aggregationFunction': '$avg'},
 '$nutrition.caloriesFromFat.amount': {'aggregate_field_address': '$nutrition.caloriesFromFat.amount',
  'aggregate_field_output_name': 'avg_nutrition_caloriesFromFat_amount',
  'aggregationFunction': '$avg'},
 '$nutrition.carbohydrates.amount': {'aggregate_field_address': '$nutrition.carbohydrates.amount',
  'aggregate_field_output_name': 'avg_nutrition_carbohydrates_amount',
  'aggregationFunction': '$avg'},
 '$nutrition.cholesterol.amount': {'aggregate_field_address': '$

In [20]:
# group bys
groupBy_fields_toQuery = [
    {
        # get global average
        'group_by_field_address': 'get_global_aggregation',
        'group_by_field_output_name': 'DummyColumn' # ignored,
    },
    {
        'group_by_field_address': '$ada-country',
        'group_by_field_output_name': 'country', # as it appears in dataframe
    },
    {
        'group_by_field_address': '$ada-city',
        'group_by_field_output_name': 'city', # as it appears in dataframe
    },
    {
        'group_by_field_address': '$ada-region',
        'group_by_field_output_name': 'region', # as it appears in dataframe
    },
]

# keep in sync with above groupBys
dataframe_names = ['global', 'per_country', 'per_city', 'per_region']

# what to aggregate for, from valid keys extracted above
aggregation_fields_toQuery = list(accepted_keys_with_arguments.values())

all_dataframes = get_dataframe_per_groupby(collection, groupBy_fields_toQuery, aggregation_fields_toQuery, dataframe_names)
print('Done')

Query 1/152 with arguments: {'group_by_field_output_name': 'DummyColumn', 'group_by_field_address': 'get_global_aggregation', 'aggregationFunction': '$avg', 'aggregate_field_address': '$nutrition.thiamin.amount', 'aggregate_field_output_name': 'avg_nutrition_thiamin_amount'}
Query 2/152 with arguments: {'group_by_field_output_name': 'DummyColumn', 'group_by_field_address': 'get_global_aggregation', 'aggregationFunction': '$avg', 'aggregate_field_address': '$similarRecipes.metaData.page', 'aggregate_field_output_name': 'avg_similarRecipes_metaData_page'}
Query 3/152 with arguments: {'group_by_field_output_name': 'DummyColumn', 'group_by_field_address': 'get_global_aggregation', 'aggregationFunction': '$avg', 'aggregate_field_address': '$nutrition.carbohydrates.amount', 'aggregate_field_output_name': 'avg_nutrition_carbohydrates_amount'}
Query 4/152 with arguments: {'group_by_field_output_name': 'DummyColumn', 'group_by_field_address': 'get_global_aggregation', 'aggregationFunction': '$a

Save them to JSON.

In [23]:
import os

json_output_filename = '../website/data/aggregatedData.json'

os.makedirs(os.path.dirname(json_output_filename), exist_ok=True)
write_dataframes_to_json(json_output_filename, all_dataframes)

In [24]:
!cat '../website/data/aggregatedData.json'

{
    "per_country": {
        "austrian": {
            "avg_nutrition_vitaminB6_amount": 0.1384612593,
            "avg_nutrition_protein_amount": 9.0687736733,
            "avg_nutrition_caloriesFromFat_amount": 131.351855,
            "avg_nutrition_fiber_amount": 1.6837512733,
            "avg_submitter_reciepesCount": 11.4666666667,
            "nbRecipes": 30,
            "avg_nutrition_folate_amount": 49.502801,
            "avg_nutrition_magnesium_amount": 27.2001603333,
            "avg_similarRecipes_metaData_pagesize": 12.7,
            "avg_nutrition_vitaminC_amount": 3.254385688,
            "avg_reviewCount": 31.6,
            "avg_nutrition_calcium_amount": 65.8696544333,
            "avg_readyInMinutes": 106.1666666667,
            "avg_nutrition_niacin_amount": 3.8091104233,
            "avg_nutrition_cholesterol_amount": 82.2841887,
            "avg_submitter_reviewsCount": 82.0333333333,
            "avg_submitter_followersCount": 27.0,
            "avg_submitter_pe

# A Safety Check for the Data

Check that the number of calories (for instance) is roughly constant per serving, i.e. the values (e.g. calories) do not need to be divided by the number of servings.

In [25]:
res = group_and_get_aggregate_of_field(
        collection, 
        group_by_field_address='$servings', 
        aggregate_field_address = '$nutrition.calories.amount', 
        group_by_field_output_name = 'servings', 
        aggregate_field_output_name = 'calories', 
        aggregationFunction = '$addToSet' 
)
res = pd.DataFrame.from_dict(res)
res['averageCalories'] = res['calories'].map(lambda x: np.mean(x))

res.head(3)

/home/maximilian/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/maximilian/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,calories,nbRecipes,servings,averageCalories
0,[130.5006],1,39,130.50060
1,[182.1069],1,26,182.10690
2,[60.56655],1,44,60.56655


In [26]:
# See how many servings are less than 20.
res[res['servings'] < 20]

,calories,nbRecipes,servings,averageCalories
27,"[30.7259, 117.7393, 409.1838, 1076.825, 165.77...",66,14,287.960521
28,"[7.663048, 114.8627, 152.9864, 96.72525, 73.0,...",752,10,327.745643
33,"[378.8421, 666.5646, 610.725, 439.9805, 472.83...",599,2,494.605193
35,"[87.89926, 58.68952, 574.4297, 10.7974, 11.35,...",501,16,238.532929
36,"[845.1532, 321.1483, 805.8989, 47.271, 381.285...",306,1,392.117609
38,"[424.8909, 397.6094, 199.6972, 217.3545, 466.8...",99,7,386.462684
40,"[120.9205, 92.85124, 152.5559, 376.8631, 280.1...",170,18,256.450726
42,"[314.1674, 290.44, 19.44138, 33.27737, 524.404...",2919,8,356.316380
43,"[469.1374, 317.9422, 582.0063, 67.08359, 414.7...",3128,6,388.387100
45,"[253.275, 24.12324, 131.3087, 227.8033, 308.63...",1526,12,298.971966


We observe there exist recipes for 240 servings !

In [27]:
anotherRecipe = collection.find_one({'servings':240})
anotherRecipe

{'_id': ObjectId('5825c738b779830612cba06d'),
 'adUnit': {'adKeys': [2,
   59,
   64,
   106,
   109,
   125,
   135,
   139,
   148,
   173,
   184,
   201,
   203,
   221,
   228,
   241,
   244,
   264,
   265,
   268,
   586,
   639,
   680],
  'adZone': 'recipe',
  'networkCode': '3865',
  'site': 'ar.ios.apps.allrecipes.recipes'},
 'ada-city': [],
 'ada-continent': [],
 'ada-country': ['us-recipes', 'australian-and-new-zealander'],
 'ada-region': ['hawaii'],
 'ada-subcontinent': ['north-american'],
 'ada-tags': ['us-recipes',
  'us-recipes-by-state',
  'hawaii',
  'world-cuisine',
  'australian-and-new-zealander'],
 'cookMinutes': 90,
 'description': 'A Hawaiian chutney, excellent with pork or lamb. Also a treat with peanut butter on bread. Note: Common mangos are small and sweet even when half-ripe, not juicy.',
 'directions': [{'displayValue': 'In a large saucepan combine vinegar, white sugar, brown sugar, cinnamon, ground ginger, allspice, cloves, nutmeg, chile peppers and sal

## Check that unit is the same for all same nutrients

There is a 'unit' field associated to each nutrient information, like 'mg', 'g', 'kg'. To avoid multiplying in mongodb, we check that they are the same across the whole database for each nutrient.

In [28]:
# Find all keys that have a unit information

all_keys_with_description = get_all_field_addresses_with_description(oneRecipe, shortenLongStrings=False)

key_addresses_with_unit = []
for key in all_keys_with_description:
    if key.endswith('.unit'):
        key_addresses_with_unit.append(key)


In [29]:
df_units = []
for keyName in key_addresses_with_unit:

    results = group_and_get_aggregate_of_field(
        collection, 
        group_by_field_address='get_global_aggregation', 
        aggregate_field_address=keyName,
        group_by_field_output_name='DummyName', 
        aggregate_field_output_name='units', 
        aggregationFunction = '$addToSet' 
    )
    
    #df_units = df_units.append(pd.DataFrame({'nutrition': make_column_name(keyName), 'observed units': results[0]['units']}))
    df_units.append({'nutrition': make_column_name(keyName), 'observed units': results[0]['units']})
    
df_units = pd.DataFrame(df_units, columns = ['nutrition' , 'observed units'])
    
df_units = df_units.reset_index()
df_units['nutrition'] = df_units['nutrition'].map(lambda elem: "avg_" + elem[:-5]) # to have the same keys as in the JSON
df_units = df_units.drop(axis=1, labels=['index'])
df_units

,nutrition,observed units
0,avg_nutrition_fiber,[g]
1,avg_nutrition_vitaminA,[IU]
2,avg_nutrition_vitaminB6,[mg]
3,avg_nutrition_potassium,[mg]
4,avg_nutrition_saturatedFat,[g]
5,avg_nutrition_niacin,[mg]
6,avg_nutrition_folate,[mcg]
7,avg_nutrition_caloriesFromFat,[kcal]
8,avg_nutrition_carbohydrates,[g]
9,avg_nutrition_cholesterol,[mg]


Hence we see that each ingredient is only given in a single unit in all recipes.

In [30]:
json_output_filename = '../website/data/nutritionAmountUnits.json'

os.makedirs(os.path.dirname(json_output_filename), exist_ok=True)
write_dataframes_to_json(json_output_filename, df_units)

!cat '../website/data/nutritionAmountUnits.json'

{
    "6": {
        "observed units": [
            "mcg"
        ],
        "nutrition": "avg_nutrition_folate"
    },
    "3": {
        "observed units": [
            "mg"
        ],
        "nutrition": "avg_nutrition_potassium"
    },
    "15": {
        "observed units": [
            "mg"
        ],
        "nutrition": "avg_nutrition_sodium"
    },
    "11": {
        "observed units": [
            "mg"
        ],
        "nutrition": "avg_nutrition_magnesium"
    },
    "19": {
        "observed units": [
            "g"
        ],
        "nutrition": "avg_nutrition_protein"
    },
    "8": {
        "observed units": [
            "g"
        ],
        "nutrition": "avg_nutrition_carbohydrates"
    },
    "12": {
        "observed units": [
            "mg"
        ],
        "nutrition": "avg_nutrition_vitaminC"
    },
    "18": {
        "observed units": [
            "g"
        ],
        "nutrition": "avg_nutrition_fat"
    },
    "16": {
        "observed units": 

# Stubs

Ignore this section.

To be removed in the very end

In [26]:
# old: not currently used
def prettifyCities(cities):
    # cities is an array of arrays of cities
    # flattens this array
    return [city for citiesList in cities for city in citiesList]

In [27]:
def getAverageOfFieldPerRegion(fieldName, fieldAddress):
    # groups by continent, subcontinent, ... and computes the average
    # fieldName used in output, fieldAddress: path to access field in the collection

    results = collection.aggregate( [ 
            { '$group': 
                {
                    '_id': { 'continent': "$ada-continent", 'subcontinent': "$ada-subcontinent", 
                            'country': "$ada-country", 'region': '$ada-region', 'city': '$ada-city'},
                    # e.g. 'avgCalories': { '$avg': '$nutrition.calories.amount' },
                    fieldName: {'$avg': ('$' + fieldAddress)}, 
                    'nbRecipes': { "$sum": 1},
                }
            },
            { '$project': 
                {
                    '_id': 0,
                    # e.g. 'avgCalories': "$avgCalories",
                    fieldName: ("$" + fieldName),
                    'nbRecipes': "$nbRecipes",
                    'continent': "$_id.continent",
                    'subcontinent': "$_id.subcontinent",
                    'country': "$_id.country",
                    'region': "$_id.region",
                    'city': "$_id.city",
                }    
            }
        ]
    )
    
    return results

results = getAverageOfFieldPerRegion(fieldName='avgCalories', fieldAddress='nutrition.calories.amount')

In [28]:
#~ # group bys
#~ groupBysToQuery = [
    #~ {
        #~ # get global average
        #~ 'group_by_field_address': 'get_global_aggregation',
        #~ 'group_by_field_output_name': 'DummyColumn' # ignored,
    #~ },
    #~ {
        #~ 'group_by_field_address': '$ada-city',
        #~ 'group_by_field_output_name': 'city', # as it appears in dataframe
    #~ },
#~ ]

#~ # keep in sync with above groupBys
#~ dataframeNames = ['global', 'per_city']

#~ # what to aggregate for
#~ fieldArgumentsToQuery = [
    #~ {
        #~ 'aggregate_field_address': '$nutrition.calories.amount', 
        #~ 'aggregate_field_output_name': 'avg_calories', 
        #~ 'aggregationFunction': '$avg' 
    #~ },
    #~ {
        #~ 'aggregate_field_address': '$nutrition.vitaminA.amount', 
        #~ 'aggregate_field_output_name': 'total_vitaminA_Intake', 
        #~ 'aggregationFunction': '$sum' 
    #~ },
#~ ]



#~ for keyName, value in allKeysWithDescription.items():
    #~ if keyName == '_id':
        #~ continue
    
    #~ res = group_and_get_aggregate_of_field(collection, group_by_field_address='get_global_aggregation',
                                    #~ aggregate_field_address=keyName, 
                                    #~ group_by_field_output_name='outputField', 
                                    #~ aggregate_field_output_name=keyName, 
                                    #~ aggregationFunction = '$type' )
    
    #~ print(res)
    #~ break
    
    #~ #get_dataframe_per_groupby(collection, groupBysToQuery, fieldArgumentsToQuery, dataframeNames)


# What is below is not yet done

Need to add one entry per row when there are multiple countries.

In [29]:
import itertools


for i, document in enumerate(results):
    fieldnames = ['continent', 'subcontinent', 'country', 'region', 'city']
    fieldVals = []
    for field in fieldnames:
        fieldVals.append(document[field] or ['NA'])
    combinations = list(itertools.product(*fieldVals))
    localDf = pd.DataFrame.from_records(combinations, columns=fieldnames)
    pd.DataFrame.add(localDf, axis=document.keys())
    
    remainingFields = list(set(document.keys()) - set(fieldnames))
    
    localDf.fillna(value={key: document[key] for key in remainingFields})
    
    if i >= 0:
        break
        
list(combinations)
localDf

TypeError: f() missing 1 required positional argument: 'other'

In [ ]:
# if there occurs an error "NoCursorFound", rerun the above query or set the property 'noCursorTimeout'
#df = pd.DataFrame(columns={'continent', 'country', 'avgCalories', 'nbRecipes'})


allData = []
for i, document in enumerate(results):
    #print(document)
    #df = df.append(document, ignore_index=True)
    allData.append(document)
    #if i >= 3:
    #    break

#allData
df = pd.DataFrame.from_dict(allData)

applyDataTransf = False

if applyDataTransf:
    joinList = lambda myList: ", ".join(myList)
    df['country'] = df['country'].apply(joinList)
    df['continent'] = df['continent'].apply(joinList)
    df['subcontinent'] = df['subcontinent'].apply(joinList)
    df['region'] = df['region'].apply(joinList)
    df['city'] = df['city'].apply(joinList)
    
    
df.head(3)

In [ ]:
for i, do

In [ ]:
df['continent'].apply(len).value_counts()

df[df['continent'].apply(len) == 2]

In [ ]:
df = df.set_index(['continent', 'subcontinent', 'country'])
df.head(3)

In [ ]:
df.groupby(level=[0,1])[['nbRecipes']].sum()

In [ ]:
# Find an element by ID
oneRecipe = collection.find_one({'recipeID':47564})
oneRecipe

In [ ]:
#started from http://stackoverflow.com/a/23282291

from bson.code import Code

# only emit the key of the field, no values
mapper = Code("""
    function() {
    
        function isObject(val) {
            if (val === null) { return false;}
            return ( (typeof val === 'function') || (typeof val === 'object') );
        }
        function getSubkeys(obj) {
            var subKeys = [];
            for (subKey in obj) {
                if (isObject(obj[subKey])) {
                    subKeys.push(subKey);
                }
                
            }
            return subKeys;
        }
        for (var key in this) { 
            subkeys = getSubkeys(this[key])
            emit(key, subkeys); 
        }
    }
""")

# keep the key, do nothing
reducer = Code("""
    function(key, values) {
        //return values.toString() + "END";
        /*
        function onlyUnique(value, index, self) { 
            return self.indexOf(value) === index;
        }
        return values.filter( onlyUnique ).toString();
        */
        return Array.from(new Set(values)).toString();
        //return new Set(values);
        
        
    }
    
    
""")

distinctThingFields = collection.map_reduce(mapper, reducer, 
    out = {'inline' : 1}, query = {'recipeID': { '$in': [47564, 98310]} }, full_response = True) #
distinctThingFields

In [ ]:
collection.aggregate( [ 
        { 
            '$project': {
                '_id': 0
            }
        }
    ]
)

In [ ]:
oneRecipe['nutrition']['sugars']

In [ ]:
distinctThingFields

In [ ]:
distinctThingFields.ok == 1.0

Resources:
* [PyMongo Tutorial](https://api.mongodb.com/python/current/tutorial.html)
* [Mongo Documentation](https://docs.mongodb.com/ecosystem/drivers/python/)